In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [2]:
from torchvision.datasets import EMNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
transform = transforms.Compose([transforms.ToTensor()])
emnist_train = EMNIST(root='./data', split='byclass', train=True, download=True, transform=transform)
emnist_test = EMNIST(root='./data', split='byclass', train=False, download=True, transform=transform)

100%|██████████| 562M/562M [00:30<00:00, 18.5MB/s]


In [4]:
def convert_dataset(dataset):
    images = []
    labels = []
    for img, label in dataset:
        images.append(img.numpy().squeeze())  # Convert to (28, 28)
        labels.append(label)
    return np.array(images), np.array(labels)

X_train, y_train = convert_dataset(emnist_train)
X_test, y_test = convert_dataset(emnist_test)


In [5]:
X_train = X_train / 255.0
X_test = X_test / 255.0

# Flatten images to 1D (ANN expects vectors)
X_train = X_train.reshape((-1, 784))
X_test = X_test.reshape((-1, 784))

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=62)
y_test = to_categorical(y_test, num_classes=62)

In [6]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),

    Dense(256, activation='relu'),
    Dropout(0.2),

    Dense(62, activation='softmax')  # 62 classes: [0-9, A-Z, a-z]
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.1)

Epoch 1/10
4908/4908 ━━━━━━━━━━━━━━━━━━━━ 92s 18ms/step - accuracy: 0.5111 - loss: 1.8285 - val_accuracy: 0.7618 - val_loss: 0.7764
Epoch 2/10
4908/4908 ━━━━━━━━━━━━━━━━━━━━ 139s 18ms/step - accuracy: 0.7559 - loss: 0.7883 - val_accuracy: 0.8025 - val_loss: 0.6108
Epoch 3/10
4908/4908 ━━━━━━━━━━━━━━━━━━━━ 143s 18ms/step - accuracy: 0.7880 - loss: 0.6589 - val_accuracy: 0.8179 - val_loss: 0.5472
Epoch 4/10
4908/4908 ━━━━━━━━━━━━━━━━━━━━ 139s 17ms/step - accuracy: 0.8024 - loss: 0.5999 - val_accuracy: 0.8259 - val_loss: 0.5124
Epoch 5/10
4908/4908 ━━━━━━━━━━━━━━━━━━━━ 141s 17ms/step - accuracy: 0.8117 - loss: 0.5633 - val_accuracy: 0.8324 - val_loss: 0.4885
Epoch 6/10
4908/4908 ━━━━━━━━━━━━━━━━━━━━ 142s 17ms/step - accuracy: 0.8187 - loss: 0.5364 - val_accuracy: 0.8379 - val_loss: 0.4704
Epoch 7/10
4908/4908 ━━━━━━━━━━━━━━━━━━━━ 142s 17ms/step - accuracy: 0.8232 - loss: 0.5218 - val_accuracy: 0.8397 - val_loss: 0.4592
Epoch 8/10
4908/4908 ━━━━━━━━━━━━━━━━━━━━ 86s 18ms/step - accuracy: 0.

In [9]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {accuracy:.4f}")

3636/3636 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.8478 - loss: 0.4320

Test Accuracy: 0.8470
